# Using a custom analyzer

This example will show you how to configure an analyzer to your own needs by selecting the modules provided by sfgad.

For this, we will create an example dataset, select a set of modules for our analyzer (including considered features and employed statistical model), train it on several iterations of the dataset, evaluate it on the last iteration and use an aggregative step to extract the most anomalous subgraph from it.

### Generate Example Dataset

Before we select and train our analyzer, we need an exemplary dataset. For this, we use one of the generators that come with the framework.

We generate a graphstream of $T = 30$ iterations with each iteration featuring the same $n = 100$ vertices. Further, the generator inserts $m = 500$ edges by randomly connecting vertices. Note, that this model does not allow loops or multiedges and generates an undirected graph.

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import datetime as dt

In [2]:
def generate_graph(n_vertices, n_edges, n_vertex_types):
    graph = nx.gnm_random_graph(n_vertices, n_edges)
    for v, data in graph.nodes(data=True):
        data['type'] = v % n_vertex_types
    for u, v, data in graph.edges(data=True):
        data['type'] = str(graph.node[u]['type']) + '_' + str(graph.node[v]['type'])
    return graph

In [3]:
DF_COLUMNS = ['TIMESTAMP', 'E_NAME', 'E_TYPE', 'SRC_NAME', 'SRC_TYPE', 'DST_NAME', 'DST_TYPE']

def from_nx(network, timestep=0):
    df = np.asarray([[dt.datetime.fromordinal(1).replace(year=2017) + dt.timedelta(days=timestep),
                      data.get('name', str((i, j))),
                      data.get('type', 'E_TYPE'),
                      str(i),
                      network.node[i].get('type', 'V_TYPE'),
                      str(j),
                      network.node[j].get('type', 'V_TYPE')] for (i, j, data) in network.edges(data=True)])
    return pd.DataFrame(data=df, columns=DF_COLUMNS)

In [4]:
def create_dataset(n_vertices, n_edges, n_vertex_types, n_timesteps):         
    return [from_nx(generate_graph(n_vertices, n_edges, n_vertex_types), timestep=i) for i in range(n_timesteps)]

In [5]:
graph_stream = create_dataset(n_vertices=100, n_edges=500, n_vertex_types=3, n_timesteps=30)

### Select Components

Every analyzer is composed of 5 different parts called components:

* The **features**
* The **selection rule** for reference observations
* The **weighting function** for selected observations
* The **estimation model** used to calculate the p-value for a feature
* The **combination method** for fusing the p-value of all features

#### Features

First, we will select the features used to describe the behavior of a vertex. These features are usually based on the graph structure, but can also rely on external sources of information.

For this example, we will use the vertex degree. It is a simple feature that is defined as the number of incident edges of a vertex. 

In [6]:
from sfgad.modules.features import VertexDegree

features = [VertexDegree()]

Note, that we wrap our feature in a list. This notation is allows use to use and compose several features to describe the behavior of a vertex.

#### Selection Rule

As the second component, we choose a selection rule for our reference observations. Reference observations are observation that used as a point of comparison to assess the anomalousness of the current behavior.

The selection rule specifies which observation to retrieve based on the meta data of a vertex. This meta data can be the name, the type, the age or any other information attached to a vertex. Further, the reference observations can be selected from current or historic observations.

In our case, we will focus on the historic observations of a vertex. This allows us to monitor and assess the individual evolution of a vertex.

In [7]:
from sfgad.modules.observation_selection import HistoricSameSelection

selection_rule = HistoricSameSelection()

Note, that it is possible to combine multiple selection rules to construct complex logical structures using the alternative, additional or fallback selection rules.

#### Weighting Function

The third component of an analyzer is the weighting function. It calculates the reliability or importance of a reference observations.

Often, the weight is defined with respect to the time difference between the current observation and the reference observation. This stems from the fact that recent observations are usually more reliable than the older ones. Alternatively, it is possible to use other sources of information such as the type to assign weights.

Note, that if all reference observations are equally reliable or important, their weights are also equal and thus have no impact on the analysis process.

For our analyzer, we will use this case and select a constant weighting function, which assign equal weights for all reference observations.

In [8]:
from sfgad.modules.weighting import ConstantWeight

weighting_function = ConstantWeight()

#### Probability Estimator

As the fourth component, we select the estimation model that is used to calculate the p-value of the current feature value. The model defines how to compare the current value to the selected reference observations.

Estimation models are either parametric or non-parametric. A parametric model assumes a specific type of distribution for a feature and estimates its parameters by fitting it to the values of the selected reference observations. Afterwards, they locate the value of the current observation and derive the corresponding p-value. Non-parametric do not assume a specific type of distribution and calculate the p-value empirically by directly comparing the current value to its reference observations.

Since the vertex degree follows an approximate Gaussian distribution, we select the corresponding parametric model.

In [9]:
from sfgad.modules.probability_estimation import Gaussian

estimator = Gaussian()

#### Probability Combiner

For the final component, we need to select a method for combining the inidivudal p-values of all features. Methods such as Fisher's method or Stouffer's method consider the distribution of these p-values and derive a staitically accurate combined p-value. However, it is also possible to simply aggregate them using methods such as the average or the minimum.

For our analyzer, we do not require a combination method since we only specified a single feature and thus have only one p-value.

In [10]:
from sfgad.modules.probability_combination import SelectedFeatureProbability

combiner = SelectedFeatureProbability()

### Initialize Analyzer

Finally, we construct our analyzer from our selected components.

In [11]:
from sfgad.analyzer import Analyzer

analyzer = Analyzer(features, selection_rule, weighting_function, estimator, combiner, n_jobs=1)

### Analyze Graphstream

In order to gain significant insight about the probability of the current state of a vertex, we need a sufficient amount of obsverations as comparison.

Since we are interested in analyzing the last iteration of the graphstream, we fit the analyzer on all but the last iteration. 

In [12]:
for g in graph_stream[:-1]:
    analyzer.fit_transform(g)

Following, we evaluate the last iteration using the fitted analyzer.

In [13]:
result = analyzer.fit_transform(graph_stream[-1])

The results show p-values ranging between 0 and 1.

In [14]:
result.sort_values(by='p_value')

,name,time_window,p_value
88,91,29,0.001296
20,20,29,0.005209
31,31,29,0.033638
67,67,29,0.035092
9,9,29,0.050420
62,62,29,0.050925
60,60,29,0.059327
94,72,29,0.061487
65,65,29,0.069826
41,41,29,0.089425


Further, we can see that the p-values roughly follow a uniform dsitribution, as we would expect.

In [15]:
result['p_value'].describe([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

count    100.000000
mean       0.498012
std        0.311865
min        0.001296
10%        0.092398
20%        0.161784
30%        0.259608
40%        0.357813
50%        0.495154
60%        0.605244
70%        0.711607
80%        0.830470
90%        0.930369
max        0.999947
Name: p_value, dtype: float64

### Extract Most Anomalous Subgraph

To extract the most anomalous (connected) subgraph, we use approaches from the aggregation module. These provide the vertices that are part of the anomaly and a score indicating its degree of anomalousness.

For our example, we use the NPHGS graph-scan with a significance threshold $\alpha_{max} = 0.2$ and $K = 5$ seed vertices.

In [16]:
from sfgad.aggregation import graph_scan

detected_vertices, score = graph_scan.scan(graph_stream[-1], result, alpha_max=0.2, K=5)

Linking the extracted vertices back to its p-values shows that are all below the singificance threshold $\alpha_{max}$.

Further, we can see that some vertices with lower p-values are not included in the extracted subgraph. This stems from the fact that they are not connected through edges to the subgraph. Vertices with higher p-values are not included since they would decrease the score of the anomaly.

In [17]:
result[result['name'].isin(detected_vertices)].sort_values('p_value')

,name,time_window,p_value
67,67,29,0.035092
62,62,29,0.050925
51,51,29,0.092729
